# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv", index_col = "City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,bilibino,68.0546,166.4372,-26.17,98,45,1.96,RU,1702246603
1,atafu village,-8.5421,-172.5159,28.53,79,96,6.98,TK,1702246605
2,port mathurin,-19.6833,63.4167,26.07,78,52,3.75,MU,1702246714
3,port-aux-francais,-49.3500,70.2167,3.56,78,56,9.91,TF,1702246575
4,avarua,-21.2078,-159.7750,27.03,78,75,11.83,CK,1702246526


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    alpha = 0.5
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# I know the challenge page had different filters than below, but that only gave 7 rows.
# I changed them to have more rows in the dataset.
city_data_df_narrowed = city_data_df.loc[
    (city_data_df["Max Temp"] < 30) &
    (city_data_df["Max Temp"] > 20) &
    (city_data_df["Wind Speed"] < 6) &
    (city_data_df["Cloudiness"] == 0),
    :
]

# Drop any rows with null values
city_data_df_narrowed = city_data_df_narrowed.dropna()

# Display sample data
city_data_df_narrowed

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
41,adamstown,-25.0660,-130.1015,21.07,58,0,4.13,PN,1702246485
96,prado,-17.3411,-39.2208,24.86,83,0,2.22,BR,1702246558
130,san carlos de bariloche,-41.1456,-71.3082,26.86,24,0,4.63,AR,1702247059
141,tazacorte,28.6290,-17.9293,21.19,44,0,4.47,ES,1702246832
225,lompoc,34.6391,-120.4579,25.08,44,0,5.66,US,1702246786
251,les avirons,-21.2418,55.3394,23.05,78,0,2.57,RE,1702247106
289,pitiquito,30.7000,-112.0833,23.94,8,0,2.87,MX,1702247122
290,cidade velha,14.9167,-23.6167,25.26,57,0,4.63,CV,1702247122
382,pisco,-13.7000,-76.2167,23.03,64,0,4.63,PE,1702246628


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_narrowed.loc[
    :,
    ["City", "Country", "Lat", "Lng", "Humidity"]
]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
41,adamstown,PN,-25.0660,-130.1015,58,
96,prado,BR,-17.3411,-39.2208,83,
130,san carlos de bariloche,AR,-41.1456,-71.3082,24,
141,tazacorte,ES,28.6290,-17.9293,44,
225,lompoc,US,34.6391,-120.4579,44,
251,les avirons,RE,-21.2418,55.3394,78,
289,pitiquito,MX,30.7000,-112.0833,8,
290,cidade velha,CV,14.9167,-23.6167,57,
382,pisco,PE,-13.7000,-76.2167,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20,
    "lang": "en"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
prado - nearest hotel: Pousada Recanto do Prado
san carlos de bariloche - nearest hotel: Kronos
tazacorte - nearest hotel: App Leyma
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
les avirons - nearest hotel: Floralys
pitiquito - nearest hotel: Hotel Oasis
cidade velha - nearest hotel: Encosta Azul
pisco - nearest hotel: La Portada
yola - nearest hotel: No hotel found
maceio - nearest hotel: Maceió Hostel e Pousada
imuris - nearest hotel: No hotel found
bell ville - nearest hotel: No hotel found
aleg - nearest hotel: Hotel Oasis Aleg
massawa - nearest hotel: Seghen International Hotel فندق دولي سيغن
gualeguay - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
41,adamstown,PN,-25.0660,-130.1015,58,No hotel found
96,prado,BR,-17.3411,-39.2208,83,Pousada Recanto do Prado
130,san carlos de bariloche,AR,-41.1456,-71.3082,24,Kronos
141,tazacorte,ES,28.6290,-17.9293,44,App Leyma
225,lompoc,US,34.6391,-120.4579,44,Embassy Suites by Hilton Lompoc Central Coast
251,les avirons,RE,-21.2418,55.3394,78,Floralys
289,pitiquito,MX,30.7000,-112.0833,8,Hotel Oasis
290,cidade velha,CV,14.9167,-23.6167,57,Encosta Azul
382,pisco,PE,-13.7000,-76.2167,64,La Portada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)